In [2]:
import os
import numpy as np
import pandas as pd
from scipy.signal import butter, filtfilt, iirnotch
import warnings
warnings.filterwarnings('ignore')

# Configuration
BASE_PATH = '/kaggle/input/mtcaic3'
OUTPUT_DIR = './preprocessed'
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Filter design
def design_filters(fs=250.0):
    # Bandpass 1-40 Hz
    bp_b, bp_a = butter(4, [1/(fs/2), 40/(fs/2)], btype='band')
    # Notch at 50 Hz
    notch_b, notch_a = iirnotch(50/(fs/2), Q=30)
    return bp_b, bp_a, notch_b, notch_a

# Preprocessing steps for one trial
def preprocess_trial(df):
    eeg_cols = ['FZ','C3','CZ','C4','PZ','PO7','OZ','PO8']
    motion_cols = ['AccX','AccY','AccZ','Gyro1','Gyro2','Gyro3']
    val_col = 'Validation'

    # 1) Motion artifact detection
    motion_mag = np.sqrt((df[motion_cols]**2).sum(axis=1))
    mot_thresh = np.percentile(motion_mag, 95)
    bad_mask = motion_mag > mot_thresh

    # 2) Mask EEG
    data = df[eeg_cols].copy().values
    data[bad_mask, :] = np.nan
    data[df[val_col] == 0, :] = np.nan

    # 3) Interpolation
    for ch in range(data.shape[1]):
        col = data[:, ch]
        nans = np.isnan(col)
        if nans.all():
            continue
        idx = np.arange(len(col))
        data[nans, ch] = np.interp(idx[nans], idx[~nans], col[~nans])

    # 4) Filtering
    bp_b, bp_a, notch_b, notch_a = design_filters()
    for ch in range(data.shape[1]):
        data[:, ch] = filtfilt(bp_b, bp_a, data[:, ch])
        data[:, ch] = filtfilt(notch_b, notch_a, data[:, ch])

    # 5) Baseline correction (first 0.5s)
    bs = int(0.5 * 250)
    baseline = data[:bs].mean(axis=0)
    data -= baseline
    return data

# Load index DataFrame
def load_index(fname, label_col=True):
    df = pd.read_csv(os.path.join(BASE_PATH, fname))
    cols = ['id','subject_id','task','trial_session','trial'] + (['label'] if label_col else [])
    return df[cols]

# Process a split with session-wise normalization and outlier removal
def process_split(df, has_label=True, split_name=''):
    data_dict = {'MI': {'X': [], 'y': [], 'id': []}, 'SSVEP': {'X': [], 'y': [], 'id': []}}
    total_trials = 0
    total_dropped = 0
    
    # Group by session
    session_groups = df.groupby(['subject_id', 'task', 'trial_session'])
    
    for (subject_id, task, trial_session), group in session_groups:
        # Load entire session data
        path = os.path.join(BASE_PATH, task, split_name, subject_id, str(trial_session), 'EEGdata.csv')
        session_df = pd.read_csv(path)
        
        # Preprocess all trials in session
        n_samp = 2250 if task == 'MI' else 1750
        session_trials = []
        valid_trial_ids = []
        
        # Process each trial in session
        for _, row in group.iterrows():
            start_idx = (row['trial'] - 1) * n_samp
            end_idx = start_idx + n_samp
            trial_data = session_df.iloc[start_idx:end_idx].copy()
            preprocessed = preprocess_trial(trial_data)
            session_trials.append(preprocessed)
            valid_trial_ids.append(row['id'])
        
        # Concatenate for session normalization
        session_data = np.concatenate(session_trials, axis=0)
        
        # Session-wise normalization
        mean = np.nanmean(session_data, axis=0)
        std = np.nanstd(session_data, axis=0)
        std[std < 1e-8] = 1.0  # Avoid division by zero
        
        # Normalize and detect outliers
        session_trials_norm = []
        outlier_flags = []
        
        for trial_idx, trial_data in enumerate(session_trials):
            # Normalize trial
            norm_trial = (trial_data - mean) / std
            
            # Calculate outlier metrics
            max_amp = np.max(np.abs(norm_trial))
            mean_var = np.mean(np.var(norm_trial, axis=0))
            
            # Thresholds (3 std from session mean)
            amp_threshold = 3 * np.std([np.max(np.abs(t)) for t in session_trials])
            var_threshold = 3 * np.std([np.mean(np.var(t, axis=0)) for t in session_trials])
            
            # Check if outlier
            is_outlier = (max_amp > amp_threshold) or (mean_var > var_threshold)
            outlier_flags.append(is_outlier)
            
            if not is_outlier:
                session_trials_norm.append(norm_trial.T)  # Transpose to (channels, time)
        
        # Update counters
        n_outliers = sum(outlier_flags)
        total_dropped += n_outliers
        total_trials += len(session_trials)
        
        if n_outliers > 0:
            print(f"Session {subject_id}/{trial_session} ({task}): Dropped {n_outliers} trials")
        
        # Store valid trials
        for idx, is_outlier in enumerate(outlier_flags):
            if not is_outlier:
                trial_id = valid_trial_ids[idx]
                data_dict[task]['X'].append(session_trials_norm.pop(0))
                data_dict[task]['id'].append(trial_id)
                if has_label:
                    label = group[group['id'] == trial_id]['label'].values[0]
                    data_dict[task]['y'].append(label)
    
    # Drop duplicates across entire split (based on ID)
    for task in data_dict:
        if data_dict[task]['id']:
            df_task = pd.DataFrame({
                'id': data_dict[task]['id'],
                'X': data_dict[task]['X'],
                'y': data_dict[task]['y'] if has_label else [None]*len(data_dict[task]['id'])
            })
            # Drop duplicate IDs
            initial_count = len(df_task)
            df_task = df_task.drop_duplicates(subset=['id'])
            dup_dropped = initial_count - len(df_task)
            if dup_dropped > 0:
                print(f"Dropped {dup_dropped} duplicates in {task} {split_name}")
                total_dropped += dup_dropped
            
            # Update data_dict
            data_dict[task]['X'] = df_task['X'].tolist()
            data_dict[task]['id'] = df_task['id'].tolist()
            if has_label:
                data_dict[task]['y'] = df_task['y'].tolist()
    
    # Stack data
    for task in data_dict:
        if data_dict[task]['X']:
            data_dict[task]['X'] = np.stack(data_dict[task]['X'])
            if has_label:
                data_dict[task]['y'] = np.array(data_dict[task]['y'])
            data_dict[task]['id'] = np.array(data_dict[task]['id'])
        else:
            data_dict[task]['X'] = np.array([])
            data_dict[task]['y'] = np.array([])
            data_dict[task]['id'] = np.array([])
    
    print(f"{split_name}: Total trials {total_trials}, Dropped {total_dropped} "
          f"({total_dropped/total_trials*100:.2f}%)")
    return data_dict

# Execute processing and save
for fname, label_col in [('train.csv', True), ('validation.csv', True), ('test.csv', False)]:
    split_name = fname.replace('.csv', '')
    df_idx = load_index(fname, label_col)
    results = process_split(df_idx, label_col, split_name=split_name)
    
    for task, d in results.items():
        out_file = f"{split_name}_{task}.npz"
        path = os.path.join(OUTPUT_DIR, out_file)
        if label_col:
            np.savez_compressed(path, X=d['X'], y=d['y'], id=d['id'])
        else:
            np.savez_compressed(path, X=d['X'], id=d['id'])
        print(f"Saved {out_file}: X={d['X'].shape}" + 
              (f", y={d['y'].shape}" if label_col and d['y'].size > 0 else "") +
              f", id={d['id'].shape}")

print('Preprocessing complete.')

train: Total trials 4800, Dropped 0 (0.00%)
Saved train_MI.npz: X=(2400, 8, 2250), y=(2400,), id=(2400,)
Saved train_SSVEP.npz: X=(2400, 8, 1750), y=(2400,), id=(2400,)
validation: Total trials 100, Dropped 0 (0.00%)
Saved validation_MI.npz: X=(50, 8, 2250), y=(50,), id=(50,)
Saved validation_SSVEP.npz: X=(50, 8, 1750), y=(50,), id=(50,)
test: Total trials 100, Dropped 0 (0.00%)
Saved test_MI.npz: X=(50, 8, 2250), id=(50,)
Saved test_SSVEP.npz: X=(50, 8, 1750), id=(50,)
Preprocessing complete.


In [11]:
import os
import numpy as np
import tensorflow as tf

from sklearn.metrics import f1_score, classification_report
from mne.decoding import CSP
from tensorflow import keras
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger, LearningRateScheduler

# --- 0) Config ---
data_dir   = './preprocessed'
output_dir = './models3'
os.makedirs(output_dir, exist_ok=True)

eeg_indices = [1, 2, 3, 4]  # C3, CZ, C4, PZ

# --- 1) Load & filter data ---
train_npz = np.load(os.path.join(data_dir, 'train_MI.npz'))
val_npz   = np.load(os.path.join(data_dir, 'validation_MI.npz'))

X_train_all, y_train = train_npz['X'], train_npz['y']
X_val_all,   y_val   = val_npz['X'],   val_npz['y']

# Filter for EEG channels only
X_train_raw = X_train_all[:, eeg_indices, :].transpose(0, 2, 1).astype('float32')  # (n, T, 4)
X_val_raw   = X_val_all[:,   eeg_indices, :].transpose(0, 2, 1).astype('float32')  # (n, T, 4)

# --- 2) Binarize labels ---
y_train_bin = (y_train == 'Right').astype(int)
y_val_bin   = (y_val   == 'Right').astype(int)

# --- 3) CSP for models 3, 4, 5 ---
csp = CSP(n_components=4, log=False, norm_trace=False)
X_train_csp_input = X_train_raw.transpose(0, 2, 1).astype('float64').copy()  # (n, channels, time)
csp.fit(X_train_csp_input, y_train_bin)  # ← NO 'rank' argument

W = csp.filters_[:4]

def apply_csp(X):  # X: (n, T, C)
    return np.stack([W.dot(ep.T) for ep in X], axis=0)


Xtr_csp = apply_csp(X_train_raw).astype('float32')
Xvl_csp = apply_csp(X_val_raw).astype('float32')

Xtr_csp = Xtr_csp.transpose(0, 2, 1)  # (n, T, 4)
Xvl_csp = Xvl_csp.transpose(0, 2, 1)


# For 2D models
Xtr_csp_2d = Xtr_csp[..., np.newaxis]
Xvl_csp_2d = Xvl_csp[..., np.newaxis]

# --- 4) One-hot labels ---
ytr_oh = keras.utils.to_categorical(y_train_bin, 2)
yvl_oh = keras.utils.to_categorical(y_val_bin,   2)

# --- 5) Data augmentation ---
def aug_gen(X, y, seed=0, batch_size=32):
    n = X.shape[0]
    rng = np.random.RandomState(seed)
    while True:
        idx = rng.randint(0, n, batch_size)
        bx, by = X[idx].copy(), y[idx]
        bx += rng.normal(0, 0.005, bx.shape)
        yield bx, by

train_gen_raw   = aug_gen(X_train_raw,  ytr_oh, seed=0, batch_size=64)
train_gen_csp1d = aug_gen(Xtr_csp,      ytr_oh, seed=1, batch_size=64)
train_gen_csp2d = aug_gen(Xtr_csp_2d,   ytr_oh, seed=2, batch_size=64)

steps_raw   = len(X_train_raw)  // 64
steps_csp1d = len(Xtr_csp)      // 64
steps_csp2d = len(Xtr_csp_2d)   // 64

# --- 6) Cosine LR schedule ---
def cosine_lr(epoch, lr_max=5e-5, epochs=200):
    return lr_max * (1 + np.cos(np.pi * epoch / epochs)) / 2

# --- 7) F1 Score Metric ---
class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name="f1_score", **kwargs):
        super().__init__(name=name, **kwargs)
        self.tp = self.add_weight(name="tp", initializer="zeros")
        self.fp = self.add_weight(name="fp", initializer="zeros")
        self.fn = self.add_weight(name="fn", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):
        preds  = tf.argmax(y_pred, axis=1)
        labels = tf.argmax(y_true, axis=1)
        self.tp.assign_add(tf.reduce_sum(tf.cast(tf.logical_and(preds == 1, labels == 1), tf.float32)))
        self.fp.assign_add(tf.reduce_sum(tf.cast(tf.logical_and(preds == 1, labels == 0), tf.float32)))
        self.fn.assign_add(tf.reduce_sum(tf.cast(tf.logical_and(preds == 0, labels == 1), tf.float32)))

    def result(self):
        p = self.tp / (self.tp + self.fp + tf.keras.backend.epsilon())
        r = self.tp / (self.tp + self.fn + tf.keras.backend.epsilon())
        return 2 * (p * r) / (p + r + tf.keras.backend.epsilon())

    def reset_states(self):
        self.tp.assign(0.0)
        self.fp.assign(0.0)
        self.fn.assign(0.0)

# --- 8) Callback factory ---
def get_callbacks(name):
    return [
        EarlyStopping("val_f1_score", mode="max", patience=20, restore_best_weights=True),
        ModelCheckpoint(os.path.join(output_dir, f"best_{name}.h5"),
                        "val_f1_score", mode="max", save_best_only=True),
        CSVLogger(os.path.join(output_dir, f"log_{name}.csv")),
        LearningRateScheduler(cosine_lr)
    ]

# --- 9) Model Builders ---
# [All model builder functions remain unchanged — see previous message if needed.]
def build_modelA(input_shape):
    m = keras.Sequential([
        layers.Input(input_shape),
        layers.Conv1D(32, 5, activation="relu", padding="same"),
        layers.BatchNormalization(), layers.MaxPool1D(2),
        layers.Conv1D(64, 5, activation="relu", padding="same"),
        layers.BatchNormalization(), layers.MaxPool1D(2),
        layers.Conv1D(128,5,activation="relu",padding="same"),
        layers.BatchNormalization(),
        layers.GlobalAveragePooling1D(),
        layers.Dense(64, activation="relu", 
                     kernel_regularizer=regularizers.l2(1e-4)),
        layers.Dropout(0.7),
        layers.Dense(2, activation="softmax"),
    ])
    m.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy", F1Score()])
    return m

def build_modelB(input_shape):
    inp = layers.Input(input_shape)
    x = inp
    for f in [16,32,64,128,256]:
        x = layers.Conv1D(f,3,activation="relu",padding="same")(x)
        x = layers.BatchNormalization()(x)
        x = layers.MaxPool1D(2)(x)
    x = layers.Flatten()(x)
    for u in [128,64,32]:
        x = layers.Dense(u, activation="relu",
                         kernel_regularizer=regularizers.l2(1e-4))(x)
        x = layers.Dropout(0.5)(x)
    out = layers.Dense(2, activation="softmax")(x)
    m = models.Model(inp, out)
    m.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy", F1Score()])
    return m

def build_model1(input_shape):
    inp = layers.Input(input_shape+(1,))
    x = layers.Concatenate()([inp, inp, inp])
    x = layers.Resizing(32,32)(x)
    base = keras.applications.ResNet50(
        include_top=False, weights="imagenet",
        input_shape=(32,32,3), pooling="avg"
    )
    base.trainable = False
    x = base(x)
    x = layers.Reshape((1, x.shape[-1]))(x)
    for _ in range(7):
        x = layers.Conv1D(64,3,activation="relu",padding="same")(x)
    x = layers.MultiHeadAttention(num_heads=4,key_dim=32)(x,x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(64,activation="relu")(x)
    out = layers.Dense(2, activation="softmax")(x)
    m = models.Model(inp,out)
    m.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy", F1Score()])
    return m

def build_model2(input_shape):
    inp = layers.Input(input_shape)
    x = inp
    for _ in range(3):
        x = layers.Conv1D(32,3,activation="elu",padding="same")(x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(64,activation="elu")(x)
    out = layers.Dense(2, activation="softmax")(x)
    m = models.Model(inp,out)
    m.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy", F1Score()])
    return m

def build_model3(input_shape):
    inp = layers.Input(input_shape)  # (T, F, 1)
    x = inp
    for _ in range(5):
        x = layers.Conv2D(32,(3,3),activation="relu",padding="same")(x)
    x = layers.Flatten()(x)
    for u in [128,64,32]:
        x = layers.Dense(u, activation="relu")(x)
    out = layers.Dense(2, activation="softmax")(x)
    m = models.Model(inp,out)
    m.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy", F1Score()])
    return m

def build_model4(input_shape):
    inp = layers.Input(input_shape)
    x = inp
    for _ in range(3):
        x = layers.Conv1D(64,3,activation="relu",padding="same")(x)
    x = layers.LSTM(128)(x)
    for _ in range(4):
        x = layers.Dense(64, activation="relu")(x)
    out = layers.Dense(2, activation="softmax")(x)
    m = models.Model(inp,out)
    m.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy", F1Score()])
    return m

def build_model5(input_shape):
    inp = layers.Input(input_shape)
    x = inp
    for _ in range(7):
        x = layers.Conv1D(64,3,activation="elu",padding="same")(x)
    x = layers.Flatten()(x)
    for _ in range(3):
        x = layers.Dense(64, activation="elu")(x)
    out = layers.Dense(2, activation="softmax")(x)
    m = models.Model(inp,out)
    m.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy", F1Score()])
    return m

def build_model6(input_shape):
    C3, C4 = 0,2
    eeg_in = layers.Input(input_shape)
    c3 = layers.Lambda(lambda x: x[:,:,C3:C3+1])(eeg_in)
    c4 = layers.Lambda(lambda x: x[:,:,C4:C4+1])(eeg_in)
    def branch():
        return models.Sequential([
            layers.Conv1D(16,250,activation="relu",padding="same"),
            layers.MaxPool1D(3),
            layers.Conv1D(32,50,activation="relu",padding="same"),
            layers.GlobalAveragePooling1D()
        ])
    b3, b4 = branch()(c3), branch()(c4)
    x = layers.Concatenate()([b3,b4])
    for _ in range(4):
        x = layers.Dense(64,activation="relu")(x)
    out = layers.Dense(2,activation="softmax")(x)
    m = models.Model(eeg_in,out)
    m.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy", F1Score()])
    return m
# (You can paste model builders here unchanged if needed)

# --- 10) Train & evaluate ---
builders = {
    'oldA': build_modelA,
    'oldB': build_modelB,
    'model1': build_model1,
    'model2': build_model2,
    'model3': build_model3,
    'model4': build_model4,
    'model5': build_model5,
    'model6': build_model6,
}

results = {}
shape_raw   = X_train_raw.shape[1:]
shape_csp1d = Xtr_csp.shape[1:]
shape_csp2d = Xtr_csp_2d.shape[1:]

for name, build_fn in builders.items():
    print(f"\n>>> Training {name}")
    if name in ['model3']:
        model = build_fn(shape_csp2d)
        gen, steps, val_x = train_gen_csp2d, steps_csp2d, Xvl_csp_2d
    elif name in ['model4', 'model5']:
        model = build_fn(shape_csp1d)
        gen, steps, val_x = train_gen_csp1d, steps_csp1d, Xvl_csp
    else:
        model = build_fn(shape_raw)
        gen, steps, val_x = train_gen_raw, steps_raw, X_val_raw

    model.fit(
        gen, steps_per_epoch=steps,
        validation_data=(val_x, yvl_oh),
        epochs=200, callbacks=get_callbacks(name), verbose=2
    )

    preds = np.argmax(model.predict(val_x), axis=1)
    f1 = f1_score(y_val_bin, preds)
    print(f"{name} → val F1 = {f1:.4f}")
    print(classification_report(y_val_bin, preds, target_names=["Left", "Right"]))
    results[name] = (f1, model)

# --- 11) Save best model ---
best_name, (best_f1, best_model) = max(results.items(), key=lambda kv: kv[1][0])
print(f"\n=== Final best: {best_name} (F1={best_f1:.4f}) ===")
best_model.save(os.path.join(output_dir, 'best_final.h5'))


Computing rank from data with rank=None
    Using tolerance 3.9 (2.2e-16 eps * 4 dim * 4.4e+15  max singular value)
    Estimated rank (data): 4
    data: rank 4 computed from 4 data channels with 0 projectors
Reducing data rank from 4 -> 4
Estimating class=0 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.

>>> Training oldA
Epoch 1/200
37/37 - 9s - 240ms/step - accuracy: 0.4882 - f1_score: 0.4925 - loss: 0.8209 - val_accuracy: 0.4200 - val_f1_score: 0.3556 - val_loss: 0.7108 - learning_rate: 0.0010
Epoch 2/200
37/37 - 1s - 27ms/step - accuracy: 0.4970 - f1_score: 0.4779 - loss: 0.7429 - val_accuracy: 0.5000 - val_f1_score: 0.1935 - val_loss: 0.7042 - learning_rate: 9.9994e-04
Epoch 3/200
37/37 - 1s - 27ms/step - accuracy: 0.5139 - f1_score: 0.4891 - loss: 0.7142 - val_accuracy: 0.5800 - val_f1_score: 0.1600 - val_loss: 0.6949 - learning_rate: 9.9969e-04
Epoch 4/200
37/37 - 1s - 28ms/step - accuracy: 0.5351 - f1_score: 0.5285 - loss: 0.7010 - val_ac

In [12]:
    import os
    import numpy as np
    from sklearn.metrics import f1_score
    import tensorflow as tf
    
    # Function to compute best threshold for a model
    def find_best_threshold(model, val_x, y_true_bin):
        probs = model.predict(val_x)[:, 1]
        thresholds = np.linspace(0.1, 1.0, 100)
        best_f1 = 0
        best_threshold = 0.5
        
        for t in thresholds:
            preds_bin = (probs > t).astype(int)
            # Use macro averaging here
            f1 = f1_score(y_true_bin, preds_bin, average='macro')  # Changed to macro
            if f1 > best_f1:
                best_f1 = f1
                best_threshold = t
                
        return best_threshold, best_f1
    
    # Load validation data
    val_npz = np.load("/kaggle/working/preprocessed/validation_MI.npz")
    X_val_all, y_val = val_npz["X"], val_npz["y"]
    y_val_bin = (y_val == "Right").astype(int)
    
    # Use only EEG channels: C3, CZ, C4, PZ
    eeg_indices = [1, 2, 3, 4]
    X_val_raw = X_val_all[:, eeg_indices, :].transpose(0, 2, 1).astype('float32')
    
    # Directories to check
    dirs = ["/kaggle/working/models3"]
    # 
    # Loop through models
    results = {}
    for model_dir in dirs:
        for file in os.listdir(model_dir):
            if file.endswith(".h5"):
                path = os.path.join(model_dir, file)
                try:
                    model = tf.keras.models.load_model(path, compile=False)
                    threshold, f1 = find_best_threshold(model, X_val_raw, y_val_bin)
                    results[path] = (threshold, f1)
                except Exception as e:
                    results[path] = f"Error: {e}"
    
    # Print results
    for model_path, (threshold, f1) in results.items():
        print(f"{model_path} → Best threshold = {threshold:.2f}, F1 score = {f1:.4f}")


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 385ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 631ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 418ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 414ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 277ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 607ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 10s 4s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 327ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 261ms/step
/kaggle/working/models3/best_model5.h5 → Best threshold = 0.52, F1 score = 0.5000
/kaggle/working/models3/best_oldB.h5 → Best threshold = 0.51, F1 score = 0.6970
/kaggle/working/models3/best_oldA.h5 → Best threshold = 0.52, F1 score = 0.5798
/kaggle/working/models3/best_model6.h5 → Best threshold = 0.50, F1 score = 0.6186
/kaggle/working/models3/best_model4.h5 → Best threshold = 0.50, F1 score = 0.5308
/kaggle/working/models3/best_final.h5 → Best threshold = 0.51, F1 score = 0.6970
/kaggle/working/models3/best_model1.h5 → Best threshold = 0.51, F1 score = 0.3590
/kaggle/working/models3/best_model3.h5 → Best threshold = 0.40, F1 score = 0.5994


In [13]:
import os
import numpy as np
from sklearn.metrics import f1_score, classification_report
import tensorflow as tf

# Function to compute best threshold and generate classification report
def evaluate_model(model, val_x, y_true_str):
    # Convert true labels to binary (Right=1, Left=0)
    y_true_bin = np.array([1 if label == "Right" else 0 for label in y_true_str])
    
    # Get prediction probabilities for class 1 (Right)
    probs = model.predict(val_x, verbose=0)[:, 1]
    
    # Find best threshold
    thresholds = np.linspace(0.1, 1.0, 100)
    best_f1 = 0
    best_threshold = 0.5
    for t in thresholds:
        preds_bin = (probs > t).astype(int)
        f1 = f1_score(y_true_bin, preds_bin)
        if f1 > best_f1:
            best_f1 = f1
            best_threshold = t
    
    # Generate predictions using best threshold
    pred_labels = ["Right" if prob > best_threshold else "Left" for prob in probs]
    
    # Generate classification report
    report = classification_report(
        y_true_str, 
        pred_labels,
        target_names=["Left", "Right"],
        digits=4
    )
    
    return best_threshold, best_f1, report

# Load validation data
val_npz = np.load("/kaggle/working/preprocessed/validation_MI.npz")
X_val_all, y_val = val_npz["X"], val_npz["y"]

# Use only EEG channels: C3, CZ, C4, PZ
eeg_indices = [1, 2, 3, 4]  # Update if your channel order differs
X_val_raw = X_val_all[:, eeg_indices, :].transpose(0, 2, 1).astype('float32')

# Directories to check
dirs = [
    "/kaggle/working/models3"
]

# Evaluate models
results = {}
for model_dir in dirs:
    for file in os.listdir(model_dir):
        if file.endswith(".h5"):
            path = os.path.join(model_dir, file)
            try:
                model = tf.keras.models.load_model(path, compile=False)
                threshold, f1, report = evaluate_model(model, X_val_raw, y_val)
                results[path] = {
                    "threshold": threshold,
                    "f1": f1,
                    "report": report
                }
            except Exception as e:
                results[path] = f"Error: {e}"

# Print results with classification reports
for path, result in results.items():
    if isinstance(result, str):
        print(f"\n{path} → {result}")
    else:
        print(f"\n{'-'*80}")
        print(f"Model: {path}")
        print(f"Best threshold: {result['threshold']:.4f}")
        print(f"Best F1-score: {result['f1']:.4f}")
        print("\nClassification Report:")
        print(result['report'])
        print(f"{'-'*80}")


--------------------------------------------------------------------------------
Model: /kaggle/working/models3/best_model5.h5
Best threshold: 0.4364
Best F1-score: 0.6197

Classification Report:
              precision    recall  f1-score   support

        Left     1.0000    0.0357    0.0690        28
       Right     0.4490    1.0000    0.6197        22

    accuracy                         0.4600        50
   macro avg     0.7245    0.5179    0.3443        50
weighted avg     0.7576    0.4600    0.3113        50

--------------------------------------------------------------------------------

--------------------------------------------------------------------------------
Model: /kaggle/working/models3/best_oldB.h5
Best threshold: 0.5000
Best F1-score: 0.6875

Classification Report:
              precision    recall  f1-score   support

        Left     1.0000    0.2857    0.4444        28
       Right     0.5238    1.0000    0.6875        22

    accuracy                        

In [24]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model

# Configuration
base_path = "/kaggle/input/mtcaic3"
preprocessed_dir = '/kaggle/working/preprocessed'  # Directory with preprocessed .npz files
model_path = '/kaggle/working/models3/best_oldA.h5'
output_file = '/kaggle/working/submission.csv'
threshold = 0.5  # Prediction threshold for MI task

# Load test data and sample submission
test_df = pd.read_csv(os.path.join(base_path, 'test.csv'))
sample_sub = pd.read_csv(os.path.join(base_path, 'sample_submission.csv'))

# Fixed F1Score metric class
class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name="f1_score", **kwargs):
        super().__init__(name=name, **kwargs)
        self.tp = self.add_weight(name="tp", initializer="zeros")
        self.fp = self.add_weight(name="fp", initializer="zeros")
        self.fn = self.add_weight(name="fn", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):
        preds = tf.argmax(y_pred, axis=1)
        labels = tf.argmax(y_true, axis=1)
        
        true_pos = tf.logical_and(tf.equal(preds, 1), tf.equal(labels, 1))
        false_pos = tf.logical_and(tf.equal(preds, 1), tf.equal(labels, 0))
        false_neg = tf.logical_and(tf.equal(preds, 0), tf.equal(labels, 1))
        
        self.tp.assign_add(tf.reduce_sum(tf.cast(true_pos, tf.float32)))
        self.fp.assign_add(tf.reduce_sum(tf.cast(false_pos, tf.float32)))
        self.fn.assign_add(tf.reduce_sum(tf.cast(false_neg, tf.float32)))

    def result(self):
        precision = self.tp / (self.tp + self.fp + tf.keras.backend.epsilon())
        recall = self.tp / (self.tp + self.fn + tf.keras.backend.epsilon())
        return 2 * (precision * recall) / (precision + recall + tf.keras.backend.epsilon())

    def reset_states(self):
        self.tp.assign(0.0)
        self.fp.assign(0.0)
        self.fn.assign(0.0)

# Load model with fixed metric
model = load_model(model_path, custom_objects={'F1Score': F1Score})

# Define EEG channels to extract and their indices in preprocessed data
channel_names = ['FZ','C3','CZ','C4','PZ','PO7','OZ','PO8']
target_channels = ['C3','CZ','C4','PZ']
target_indices = [channel_names.index(ch) for ch in target_channels]

# Load preprocessed test data
test_mi = np.load(os.path.join(preprocessed_dir, 'test_MI.npz'))['X']  # Shape (n_mi, 8, 2250)
test_ssvep = np.load(os.path.join(preprocessed_dir, 'test_SSVEP.npz'))['X']  # Shape (n_ssvep, 8, 1750)

# Extract target channels and transpose to (trials, time, channels)
test_mi = test_mi[:, target_indices, :].transpose(0, 2, 1)  # New shape: (n_mi, 2250, 4)
test_ssvep = test_ssvep[:, target_indices, :].transpose(0, 2, 1)  # New shape: (n_ssvep, 1750, 4)

# Get indices of MI and SSVEP trials in test_df
mi_mask = test_df['task'] == 'MI'
ssvep_mask = test_df['task'] == 'SSVEP'

# Validate counts
assert mi_mask.sum() == test_mi.shape[0], "MI trial count mismatch"
assert ssvep_mask.sum() == test_ssvep.shape[0], "SSVEP trial count mismatch"

# Generate predictions
predictions = []
# Predict MI trials
if test_mi.shape[0] > 0:
    mi_preds = model.predict(test_mi, verbose=0)
    mi_labels = ['Right' if prob[1] >= threshold else 'Left' for prob in mi_preds]
    predictions.extend(mi_labels)

# Assign dummy labels for SSVEP trials
if test_ssvep.shape[0] > 0:
    predictions.extend(['Left'] * test_ssvep.shape[0])

# Create submission file
submission = sample_sub.copy()
submission['label'] = predictions
submission.to_csv(output_file, index=False)

print(f"Submission file saved to {output_file}")
print("Prediction distribution:")
print(submission['label'].value_counts())
print(f"\nThreshold used for MI: {threshold}")

Submission file saved to /kaggle/working/submission.csv
Prediction distribution:
label
Left     59
Right    41
Name: count, dtype: int64

Threshold used for MI: 0.5
